In [ ]:
from rdkit import Chem
import pandas as pd
import numpy as np
import glob
import os
from rdkit import DataStructs
from rdkit.Chem import rdMolDescriptors,MolStandardize,AllChem,rdMolDescriptors
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt

import random
random.seed(100)

In [ ]:
data = np.array(pd.read_csv('**.csv')) # put your own data path
kf = KFold(n_splits=5,shuffle=True,random_state=100)

In [ ]:
# calculate molecular fingerprint
def get_fps(smi_list):
    fps = []
    for i in range(len(smi_list)):
        mol = Chem.MolFromSmiles(MolStandardize.standardize_smiles(smi_list[i]))
        fps2 = AllChem.GetMorganFingerprintAsBitVect(mol,4,nBits=500)
        fps.append(fps2)
    fps = np.array(fps)
    return fps

In [ ]:
# calculate the uncertainty
def get_uncertainty(reg,x_train,train_smiles,train_labels,train_predict,x_test,test_smiles,test_labels,test_predict,scaler):
    child_estimators = reg.estimators_
    train_estimators_results = []
    for i in range(len(child_estimators)):
        estimator = child_estimators[i]
        r = estimator.predict(x_train).reshape(-1,1)
        train_estimators_results.append(r)            
    train_estimators_results = scaler.inverse_transform(np.concatenate(train_estimators_results,axis=1))
    std = np.array(np.std(train_estimators_results,axis=1)).reshape(-1,1)       
    train_results = np.concatenate([np.array(train_smiles).reshape(-1,1),np.array(train_labels),train_predict,std],axis=1)
    test_estimators_results = []
    for i in range(len(child_estimators)):
        estimator = child_estimators[i]
        r = estimator.predict(x_test).reshape(-1,1)
        test_estimators_results.append(r)            
    test_estimators_results = scaler.inverse_transform(np.concatenate(test_estimators_results,axis=1))
    test_std = np.array(np.std(test_estimators_results,axis=1)).reshape(-1,1)       
    test_results = np.concatenate([np.array(test_smiles).reshape(-1,1),np.array(test_labels),test_predict,test_std],axis=1)

    M_std = np.median(std)      
    record = []    
    for row in std:
        record.append(np.array(abs(row[0]-M_std)).reshape(-1,1)) 
    record = np.concatenate(record,axis=0)          
    mad = 1.4826*np.median(record)                  
    d_c = M_std+3*mad                              
    new_train_data = []
    culled_train_data = []
    for row in train_results:
        if float(row[3]) <= d_c:       
            new_train_data.append(row.reshape(1,-1))
        else:
            culled_train_data.append(row.reshape(1,-1))     
    new_train_data = np.concatenate(new_train_data,axis=0)
    if len(culled_train_data) != 0:
        culled_train_data = np.concatenate(culled_train_data,axis=0)
    else:
        print('no molecule removed')
    new_test_data = []
    culled_test_data = []
    for row in test_results:
        if float(row[3]) <= d_c:     
            new_test_data.append(row.reshape(1,-1))
        else:
            culled_test_data.append(row.reshape(1,-1))
    try:
        new_test_data = np.concatenate(new_test_data,axis=0)
    except:
        pass
    if len(culled_test_data) != 0:
        culled_test_data = np.concatenate(culled_test_data,axis=0)
    else:
        print('no molecule removed')
    return new_train_data,new_test_data,culled_train_data,culled_test_data,M_std,d_c,mad


# set multiple iteration in each fold
def uncertainty_loop(j,loop,train_data,test_data):    
    M_std_record , d_c_record ,mad_record = [] , [] , []        
    train_data_record , test_data_record = [] , []
    culled_train_record , culled_test_record = [] , []
    acc_record = []                                                         
    train_smiles , test_smiles = list(train_data.iloc[:,0]),list(test_data.iloc[:,0])       
    train_labels , test_labels = np.array(train_data.iloc[:,1]).reshape(-1,1),np.array(test_data.iloc[:,1]).reshape(-1,1)
    train_scaler = StandardScaler().fit(train_labels.reshape(-1,1))
    normed_train = train_scaler.transform(train_labels.reshape(-1,1))          
    uncertainty_result = pd.DataFrame()
    for i in range(loop):  
        x_train,x_test = get_fps(train_smiles),get_fps(test_smiles)       

        reg = RandomForestRegressor(n_jobs=30)  
        reg  = reg.fit(x_train,normed_train.ravel())
        train_predict = train_scaler.inverse_transform(reg.predict(x_train).reshape(-1,1))
        test_predict = train_scaler.inverse_transform(reg.predict(x_test).reshape(-1,1))
        print('-'*10,i,'-'*10)

        r2_train , r2_test = r2_score(train_labels,train_predict) , r2_score(test_labels,test_predict)
        mae_train , mae_test = mean_absolute_error(train_labels,train_predict) , mean_absolute_error(test_labels,test_predict)
        mse_train , mse_test = mean_squared_error(train_labels,train_predict) , mean_squared_error(test_labels,test_predict)

        acc_record.append([r2_train,mae_train,mse_train,r2_test,mae_test,mse_test])
        print(r2_train,mae_train,mse_train,'\n',r2_test,mae_test,mse_test)

        new_train_data,new_test_data,culled_train_data,culled_test_data,M_std,d_c,mad = get_uncertainty(reg,x_train,train_smiles,train_labels,train_predict,x_test,test_smiles,test_labels,test_predict,scaler=train_scaler)

        print('M',M_std,'MAD',mad,'threshold',d_c)
        try:
            print('the number of training molecules',new_train_data.shape[0],'the number of test molecules',new_test_data.shape[0])
        except:
            print('warning: new dataset has no data...')

        train_data_record.append(new_train_data)        
        test_data_record.append(new_test_data)
        culled_train_record.append(culled_train_data)
        culled_test_record.append(culled_test_data)

        para = np.array([M_std,mad,d_c])
        M_std_record.append(M_std)      
        d_c_record.append(d_c)   
        mad_record.append(mad) 


        train_smiles , test_smiles = list(new_train_data[:,0]),list(new_test_data[:,0])
        train_labels , test_labels = np.array(new_train_data[:,1],dtype=np.float32).reshape(-1,1),np.array(new_test_data[:,1],dtype=np.float32).reshape(-1,1)
        train_scaler = StandardScaler().fit(train_labels.reshape(-1,1))       
        normed_train = train_scaler.transform(train_labels.reshape(-1,1))      

    return M_std_record,d_c_record,mad_record,train_data_record,test_data_record,culled_train_record,culled_test_record,acc_record

In [ ]:
train_data_cv = []
test_data_cv = []

culled_train_cv = []
culled_test_cv = []

M_std_cv = []
d_c_cv = []
mad_c_cv = []

acc_cv = []

f=1
for train, test in kf.split(data):

    print('*'*50,f'{f}fold','*'*50)
    train_data = pd.DataFrame(data[train])
    test_data = pd.DataFrame(data[test])


    M_std_record,d_c_record,mad_record,train_data_record,test_data_record,culled_train_record,culled_test_record,acc_record = uncertainty_loop(f,15,train_data,test_data)

    culled_train_cv.append(culled_train_record)
    culled_test_cv.append(culled_test_record)


    train_data_cv.append(train_data_record[-1])
    test_data_cv.append(test_data_record[-1])


    acc_cv.append(acc_record)


    M_std_cv.append(M_std_record)
    d_c_cv.append(d_c_record)
    mad_c_cv.append(mad_record)
    
    f+=1

In [ ]:
# 5 - fold
test1 = test_data_cv[0]
test2 = test_data_cv[1]
test3 = test_data_cv[2]
test4 = test_data_cv[3]
test5 = test_data_cv[4]

culled_test1 = culled_test_cv[0]
culled_test1 = [elem for elem in culled_test1 if len(elem) > 0]
culled_test1 = np.concatenate(culled_test1)

culled_test2 = culled_test_cv[1]
culled_test2 = [elem for elem in culled_test2 if len(elem) > 0]
culled_test2 = np.concatenate(culled_test2)

culled_test3 = culled_test_cv[2]
culled_test3 = [elem for elem in culled_test3 if len(elem) > 0]
culled_test3 = np.concatenate(culled_test3)

culled_test4 = culled_test_cv[3]
culled_test4 = [elem for elem in culled_test4 if len(elem) > 0]
culled_test4 = np.concatenate(culled_test4)

culled_test5 = culled_test_cv[4]
culled_test5 = [elem for elem in culled_test5 if len(elem) > 0]
culled_test5 = np.concatenate(culled_test5)


all_culled_data = list(list(culled_test1[:,0])+list(culled_test2[:,0])+list(culled_test3[:,0])+list(culled_test4[:,0])+list(culled_test5[:,0])
all_culled_data = list(set(all_culled_data))       
new_data = []       
for row in data:
    if row[0] not in all_culled_data:
        new_data.append(row.reshape(1,-1))

new_data = np.concatenate(new_data,axis=0)

pd.DataFrame(new_data).to_csv('**_washed.csv',index=False) # put your own data path  ; 'new_data' is the final high-quality dataset